In [1]:
!pip install -q ultralytics opencv-python-headless


In [2]:
import os
os.environ["YOLO_VERBOSE"] = "False"

import cv2
from ultralytics import YOLO
from collections import deque
from IPython.display import Video


In [3]:
VIDEO_PATH = "/kaggle/input/volley/volleyball_match.mp4"
OUTPUT_PATH = "/kaggle/working/volleyball.mp4"


In [4]:
model = YOLO("yolov8n.pt")


In [5]:
cap = cv2.VideoCapture(VIDEO_PATH)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height)
)


In [6]:
trajectory = deque(maxlen=30)


In [7]:
import numpy as np

# ---- DEFINE COURT POLYGON (ADJUST ONCE IF NEEDED) ----
court_polygon = np.array([
    (int(0.25 * width), int(0.22 * height)),   # top-left
    (int(0.75 * width), int(0.22 * height)),   # top-right
    (int(0.92 * width), int(0.78 * height)),   # bottom-right
    (int(0.08 * width), int(0.78 * height))    # bottom-left
], dtype=np.int32)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_blur = cv2.GaussianBlur(frame, (5, 5), 0)
    results = model(frame_blur, conf=0.4, verbose=False)[0]

    left_count = 0
    right_count = 0

    for box in results.boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = model.names[cls]

        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2

        # ---------- PLAYER (INSIDE COURT ONLY) ----------
        if label == "person":
            # pointPolygonTest returns >=0 if point is inside polygon
            if cv2.pointPolygonTest(court_polygon, (cx, cy), False) >= 0:

                if cx < width // 2:
                    left_count += 1
                else:
                    right_count += 1

                cv2.rectangle(frame, (x1, y1), (x2, y2),
                              (0, 255, 0), 2)

        # ---------- BALL ----------
        elif label == "sports ball":
            center = ((x1 + x2)//2, (y1 + y2)//2)
            trajectory.append(center)

            radius = max(8, (x2 - x1)//2)
            cv2.circle(frame, center, radius, (0, 0, 255), 2)
            cv2.circle(frame, center, 3, (0, 0, 255), -1)

    # ---------- BALL TRAJECTORY ----------
    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i-1], trajectory[i],
                 (0, 0, 255), 2)

    # ---------- DRAW COURT (OPTIONAL, FOR DEBUG) ----------
    cv2.polylines(frame, [court_polygon], True, (255, 255, 0), 2)

    # ---------- MIDLINE ----------
    cv2.line(frame, (width//2, 0), (width//2, height),
             (255, 255, 255), 2)

    # ---------- COUNTS ----------
    cv2.putText(frame, f"Left: {left_count}",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255,255,255), 2)

    cv2.putText(frame, f"Right: {right_count}",
                (width - 200, 40), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255,255,255), 2)

    out.write(frame)


In [8]:
cap.release()
out.release()


In [ ]:
Video(OUTPUT_PATH, embed=True)
